In [ ]:
pip install memory_profiler

In [ ]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from memory_profiler import memory_usage
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier, StackingClassifier
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import EfficientNetB7
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import psutil
# import resource
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve, auc
from sklearn.model_selection import train_test_split

In [ ]:
# Define the image size
import os
IMG_SIZE = 50

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab-Notebooks/Anomaly-Detection/CSV

# train_df = pd.read_csv('./train_final_index.csv')
# test_df = pd.read_csv('./test_final_index.csv')
# test_df = pd.read_csv('./val_final_index.csv')
train_df = pd.read_csv('./train_feature_index.csv')
# test_df = pd.read_csv('./test_feature_index.csv')
# test_df = pd.read_csv('./val_feature_index.csv')

path = "/content/drive/MyDrive/Colab-Notebooks/Anomaly-Detection/fiveTwelve"
print("found", os.path.exists(path=path))


Mounted at /content/drive
/content/drive/MyDrive/Colab-Notebooks/Anomaly-Detection/CSV
found True


In [ ]:
X_train = train_df.drop(columns=["label", "path"])
y_train = train_df["label"]

X_test = train_df.drop(columns=["label", "path"])
y_test = train_df["label"]

def load_and_preprocess_image(path):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    image = image / 255.0
    return image


In [ ]:
#Remove the memory code after testing
# Sample 1
process = psutil.Process()
before_memory = process.memory_info().rss / 1024 / 1024  # Convert to MB
print("process", process, "before Memory", before_memory)


process psutil.Process(pid=650, name='python3', status='running', started='12:53:43') before Memory 607.3515625


In [ ]:
#KNN
X_train_reorder = np.random.permutation(X_train)
knn = KNeighborsClassifier()
params_knn = {"n_neighbors": np.arange(1, 25)}
knn_gs = GridSearchCV(knn, params_knn, cv=10)
knn_gs.fit(X_train, y_train)
knn_gs.fit(X_train_reorder, y_train)

best_params = knn_gs.best_params_
best_score = knn_gs.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

used_algorithm = knn.algorithm

knn_best = knn_gs.best_estimator_
print(knn_gs.best_params_)


Best Parameters: {'n_neighbors': 6}
Best Score: 0.9788868827096591
{'n_neighbors': 6}


In [ ]:
#Random Forest
X_train_reorder = np.random.permutation(X_train)
rf = RandomForestClassifier()
params_rf = {"n_estimators": [50, 100, 200]}
rf_gs = GridSearchCV(rf, params_rf, cv=10)
rf_gs.fit(X_train, y_train)
rf_gs.fit(X_train_reorder, y_train)


rf_best = rf_gs.best_estimator_
print(rf_gs.best_params_)
rf_best_params = rf_gs.best_params_
rf_best_score = rf_gs.best_score_

print("Best Parameters:", rf_best_params)
print("Best Score:", rf_best_score)


{'n_estimators': 50}
Best Parameters: {'n_estimators': 50}
Best Score: 0.9788653634499218


In [ ]:
print("knn: {}".format(knn_best.score(X_test, y_test)))
print("rf: {}".format(rf_best.score(X_test, y_test)))

estimators=[("knn", knn_best), ("rf", rf_best)]
ensemble = VotingClassifier(estimators, voting="hard")
# ensemble = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

ensemble.fit(X_train, y_train)

ens_best = ensemble.score(X_test, y_test)

print("Ens_best", ens_best)

y_pred = ensemble.predict(X_test)

cm = confusion_matrix(y_test, y_pred)

print("confusion_matrix", cm)

y_pred = ensemble.predict(X_test)

y_pred_labels = y_pred

f1 = f1_score(y_test, y_pred_labels, average='macro')
print("F1 Score:", f1)

ens_labels = ensemble.predict(X_test)

score1 = knn_best.score(X_test, y_test)
score2 = rf_best.score(X_test, y_test)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


knn: 0.9788868801652892


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


rf: 0.959323347107438
Ens_best 0.9856017561983471
confusion_matrix [[45483     0]
 [  669   312]]
F1 Score: 0.7376489520045546


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [ ]:
def image_generator(X, y, batch_size=32):
    num_samples = len(X)
    indices = np.arange(num_samples)
    while True:
        np.random.shuffle(indices)
        batch_indices = []
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)
            batch_indices.extend(indices[start:end])

            if len(batch_indices) == batch_size:
                batch_X = np.zeros((batch_size, 224, 224, 3))
                batch_y = np.zeros((batch_size,))  # Use num_classes from your problem

                for i, idx in enumerate(batch_indices):
                    if idx >= num_samples:
                        continue  # Skip this sample

                    if "path" not in X.iloc[idx]:
                        continue  # Skip this sample if 'path' key is missing

                    path = X.iloc[idx]["path"]
                    image = load_and_preprocess_image(path)
                    if image is None:
                        continue  # Skip this sample

                    batch_X[i] = image
                    batch_y[i] = y.iloc[idx]

                yield batch_X, batch_y
                batch_indices = []

        # If there are remaining samples that don't form a full batch, process them as well
        if batch_indices:
            batch_X = np.zeros((len(batch_indices), 224, 224, 3))
            batch_y = np.zeros((len(batch_indices), 10))  # Use num_classes from your problem
            for i, idx in enumerate(batch_indices):
                if idx >= num_samples:
                    continue  # Skip this sample

                if "path" not in X.iloc[idx]:
                    continue  # Skip this sample if 'path' key is missing

                path = X.iloc[idx]["path"]
                image = load_and_preprocess_image(path)
                if image is None:
                    continue  # Skip this sample

                batch_X[i] = image
                batch_y[i] = y.iloc[idx]

            yield batch_X, batch_y

def load_and_preprocess_image(path):
    image = cv2.imread(path)
    if(image is not None):
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      image = cv2.resize(image, (224, 224))
      image = image / 255.0
    return image

# X_train = train_df.drop(columns=["label", "path_id"])
# y_train = train_df["label"]

# X_test = test_df.drop(columns=["label", "path_id"])
# y_test = test_df["label"]
X = train_df.drop(columns=["label", "path_id"])
y = train_df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train:", X_train)
print("X_test:", X_test)
print("y_train:", y_train)
print("y_test:", y_test)

X_train:             radius    texture     perimeter    area    smoothness  \
40320   217.524111  76.482487   1366.744299   75391  2.477736e+01   
4235   1569.689986  44.668763   9862.653060       2  4.863596e+07   
8723   1523.988731  46.390498   9575.503602   87264  1.050723e+03   
15482  1324.938432  40.284425   8324.833688   99147  6.989910e+02   
15541   221.983998  71.378539   1394.766594  101983  1.907547e+01   
...            ...        ...           ...     ...           ...   
11284  2440.103466  36.714284  15331.622250  102674  2.289369e+03   
44732  1165.958112  48.651823   7325.930876   94176  5.698826e+02   
38158  1835.768665  44.191267  11534.474700   96081  1.384708e+03   
860     209.415609  76.584200   1315.797077  130988  1.321741e+01   
15795  1970.412914  55.397512  12380.469470   93826  1.633620e+03   

        compactness  concavity  concave_points   symmetry  fractal_dimension  \
40320  2.477736e+01       4801          -79680  39.101562           0.952986   
42

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
def run_code():
    base_model = EfficientNetB7(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
    base_model.trainable = False

    inputs = keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    outputs = keras.layers.Dense(10, activation="softmax")(x)
    model = keras.Model(inputs, outputs)
    optimizer = keras.optimizers.Adam(learning_rate=0.01)
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    print("***********Model Complied*************")
    # print("X_test", X_test, "y_test", y_test)
    train_generator = image_generator(X_train, y_train, 500)
    test_generator = image_generator(X_test, y_test, 500)
    # print("test_generator",test_generator)

    X_train_processed, y_train_processed = next(train_generator)
    X_test_processed, y_test_processed = next(test_generator)
    # print("X_test_processed", len(X_test_processed), "y_test_processed", len(y_test_processed))

    epochs = 10
    accuracy_history = []
    loss_history = []
    val_loss_history=[]
    val_accuracy_history=[]

    for epoch in range(epochs):
        history = model.fit(X_train_processed, y_train_processed, epochs=1, validation_data=(X_test_processed, y_test_processed))
        batch_size = 32
        training_accuracy = history.history['accuracy'][0]
        training_loss = history.history['loss'][0]
        val_accuracy=history.history['val_accuracy'][0]
        val_loss=history.history['val_loss'][0]

        # Append the accuracy and loss lists for all epochs to history
        accuracy_history.append(training_accuracy)
        loss_history.append(training_loss)
        val_accuracy_history.append(val_accuracy)
        val_loss_history.append(val_loss)

    # model.fit(X_train_processed, y_train_processed, validation_data=(X_test_processed, y_test_processed), epochs=10)

    print("accuracy_history:", accuracy_history)
    print("loss_history:", loss_history)
    print("***********Completed*************")
    print("val_loss_history:", val_loss_history)
    print("val_accuracy_history:", val_accuracy_history)
    test_loss, test_acc = model.evaluate(X_test_processed, y_test_processed)
    print("Test Loss:",test_loss, "Test accuracy:", test_acc)
    print("***********Completed*************")
    # Assuming y_test_processed contains binary labels (0 and 1)
    y_pred = model.predict(X_test_processed)
    y_pred_labels = y_pred.argmax(axis=1) # Convert probabilities to binary labels

    # print("y_test_processed", y_test_processed , "y_pred_labels", y_pred_labels, "X_test_processed", X_test_processed)

    # Generate classification report
    report = classification_report(y_test_processed, y_pred_labels)
    print(report)
    print("************************************")
    # Convert the data type of y_test_processed to integer
    y_test_processed = y_test_processed.astype(int)

    # Get predicted labels from the model
    y_pred = model.predict(X_test_processed)
    print("y_pred", y_pred, y_test_processed)
    y_pred_labels = y_pred.argmax(axis=1)

    # Generate the confusion matrix
    cm = confusion_matrix(y_test_processed, y_pred_labels)
    print("Confusion Matrix:", cm)
    print("************************************")
    # Calculate the F1 score
    f1 = f1_score(y_test_processed, y_pred_labels, average='macro')
    print("F1 Score:", f1)
    print("************************************")

    #Accuracy Graph
    plt.figure(figsize=(8, 6))
    # Plot training accuracy
    plt.plot(range(1, epochs + 1), np.exp(accuracy_history), linestyle='-', label='Training Accuracy')
    # Plot testing accuracy
    plt.plot(range(1, epochs + 1), np.exp(val_accuracy_history), linestyle='-', label='Testing Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.grid(True)
    plt.xticks(np.arange(1, epochs + 1))
    plt.ylim(0, np.exp(1))
    plt.legend()
    plt.show()

    #Loss Graph
    plt.figure(figsize=(8, 6))
    # Plot training loss
    plt.plot(range(1, epochs + 1), np.exp(loss_history), linestyle='-', label='Training Loss')
    # Plot testing loss
    plt.plot(range(1, epochs + 1), np.exp(val_loss_history), linestyle='-', label='Testing Loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.xticks(np.arange(1, epochs + 1))
    plt.ylim(0, np.exp(1))
    plt.legend()
    plt.show()

    # Precision-Recall Curve
    precision, recall, thresholds = precision_recall_curve(y_test_processed, y_pred[:, 1])

    # Calculate area under the precision-recall curve (AUC-PR)
    auc_pr = auc(recall, precision)
    print("Precision-Recall curve: AUC=", auc_pr)

    # Plot precision-recall curve
    plt.figure()
    plt.plot(recall, precision, color='blue', lw=2, label='Precision-Recall curve (AUC = %0.2f)' % auc_pr)
    plt.plot(recall, precision, color='blue', lw=2, label='Precision-Recall curve')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc='lower left')
    plt.show()
    return 1


In [ ]:
run_code()

after_memory = process.memory_info().rss / 1024 / 1024  # Convert to MB
memory_usage = after_memory - before_memory
print("Memory usage:", memory_usage, "MB")

NameError: ignored